In [4]:
import d2lzh as d2l
import math
from mxnet import autograd, gluon, init, nd
from mxnet.gluon import loss as gloss, nn, rnn
import time
import zipfile

ctx = d2l.try_gpu()
print('will use', ctx)

will use cpu(0)


In [5]:
# load dataset
def load_data_jay_lyrics():
    """Load the Jay Chou lyric data set (available in the Chinese book)."""
    with zipfile.ZipFile('data/jaychou_lyrics.txt.zip') as zin:
        with zin.open('jaychou_lyrics.txt') as f:
            corpus_chars = f.read().decode('utf-8')
    corpus_chars = corpus_chars.replace('\n', ' ').replace('\r', ' ')
    corpus_chars = corpus_chars[0:10000]
    idx_to_char = list(set(corpus_chars))
    char_to_idx = dict([(char, i) for i, char in enumerate(idx_to_char)])
    vocab_size = len(char_to_idx)
    corpus_indices = [char_to_idx[char] for char in corpus_chars]
    return corpus_indices, char_to_idx, idx_to_char, vocab_size

(corpus_indices, char_to_idx, idx_to_char, vocab_size) = load_data_jay_lyrics()

In [6]:
vocab_size

1027

定义模型

In [37]:
num_hiddens = 256
rnn_layer = rnn.RNN(num_hiddens)
rnn_layer.initialize()

In [39]:
batch_size = 2
state = rnn_layer.begin_state(batch_size = batch_size)
state[0].shape

(1, 2, 256)

In [43]:
# INPUT: （时间步数，batch_size，输入个数（vocab_size））

# OUTPUT：
#     Y: （时间步数，batch_size，隐藏单元个数），未通过输出层计算
#     State（隐藏状态）

num_steps = 40
X = nd.random.uniform(shape = (num_steps, batch_size, vocab_size))
Y, state_new = rnn_layer(X, state)
print(Y.shape, len(state_new), state_new[0].shape)

Y.reshape((-1, Y.shape[-1])).shape

(40, 2, 256) 1 (1, 2, 256)


(80, 256)

In [44]:
# 继承Block类来定义RNN
class RNNModel(nn.Block):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.dense = nn.Dense(vocab_size)
        
    def forward(self, input, state):
        # One hot encoding; use tranpose to get the right dimension (num_steps, batch_size)
        X = nd.one_hot(input.T, self.vocab_size)
        Y, state = self.rnn(X, state)
        # 然后是输出层：
        # 全连接层会首先将Y的形状变成(num_steps * batch_size, num_hiddens)，
        # 它的输出形状为(num_steps * batch_size, vocab_size)
        output = self.dense(Y.reshape((-1 ,Y.shape[-1])))
        return output, state
    
    def begin_state(self, *args, **kwargs):
        return self.rnn.begin_state(*args, **kwargs)

训练模型

In [54]:
def predict_rnn_gluon(prefix, num_chars, model, vocab_size, ctx, idx_to_char,
                      char_to_idx):
    state = model.begin_state(batch_size=1, ctx=ctx)
    output = [char_to_idx[prefix[0]]]
    
    for t in range(len(prefix) + num_chars - 1):
        X = nd.array([output[-1]], ctx=ctx).reshape((1,1))
        Y, state = model(X, state)
        
        if t < len(prefix)-1:
            output.append(char_to_idx[prefix[t+1]])
        else:
            output.append(int(Y.argmax(axis = 1).asscalar()))
    return ''.join([idx_to_char[i] for i in output])

In [57]:
ctx = d2l.try_gpu()
model = RNNModel(rnn_layer, vocab_size)
model.initialize(force_reinit=True, ctx=ctx)
predict_rnn_gluon('他们', 15, model, vocab_size, ctx, idx_to_char, char_to_idx)

'他们将彻明外甘杂将果优拳承鼻纯岁彻'

In [62]:
def train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                                corpus_indices, idx_to_char, char_to_idx,
                                num_epochs, num_steps, lr, clipping_theta,
                                batch_size, pred_period, pred_len, prefixes):
    loss = gloss.SoftmaxCrossEntropyLoss()
    model.initialize(ctx=ctx, force_reinit=True, init=init.Normal(0.01))
    trainer = gluon.Trainer(model.collect_params(), 'sgd', 
                           {'learning_rate': lr, 'momentum': 0, 'wd': 0})
    
    for epoch in range(num_epochs):
        l_sum, n, start = 0.0, 0, time.time()
        data_iter = d2l.data_iter_consecutive(
             corpus_indices, batch_size, num_steps, ctx)
        state = model.begin_state(batch_size=batch_size, ctx=ctx)
        
        for X,Y in data_iter:
            for s in state:
                s.detach()
            with autograd.record():
                (output, state) = model(X, state)
                y = Y.T.reshape((-1,))
                l = loss(output, y).mean()
            l.backward()
            # clipping grad
            params = [p.data() for p in model.collect_params().values()]
            d2l.grad_clipping(params, clipping_theta, ctx)
            trainer.step(1)
            l_sum += l.asscalar() * y.size
            n += y.size
            
        if (epoch + 1) % pred_period == 0:
            print('epoch %d, perplexity %f, time %.2f sec' % (
                epoch + 1, math.exp(l_sum / n), time.time() - start))
            for prefix in prefixes:
                print(' -', predict_rnn_gluon(
                    prefix, pred_len, model, vocab_size, ctx, idx_to_char,
                    char_to_idx))

In [74]:
num_epochs, batch_size, lr, clipping_theta = 500, 32, 1e2, 1e-2
pred_period, pred_len, prefixes = 100, 100, ['我是谁', '是谁']
train_and_predict_rnn_gluon(model, num_hiddens, vocab_size, ctx,
                            corpus_indices, idx_to_char, char_to_idx,
                            num_epochs, num_steps, lr, clipping_theta,
                            batch_size, pred_period, pred_len, prefixes)

epoch 100, perplexity 21.159455, time 0.62 sec
 - 我是谁烦错多 我不要你的你笑每天都能看要在有 你来一定是我 不知这进队人已的一头在缓著一等 说不我有轻你是一场悲剧伤你的那袋 让有在木不 我不能再想 我不能我想你睡爱火 是的的美 你来一定开我 不知这进队人
 - 是谁 娘子的美蜜在 一直在老 快人了沙凉棍 哼哼的客栈人多的老没有开 爱可的让我感动的可爱女人 透柔的让我疯狂的可爱女人 透柔的让我疯狂的可爱女人 透柔的让我疯狂的可爱女人 透柔的让我疯狂的可爱女人 透柔
epoch 200, perplexity 2.915417, time 0.60 sec
 - 我是谁处难躲 我 上你说 分子我娘已踢 飞檐走壁游药 快攻抢篮板都 得分都靠板都 一直都靠板 不天是乌鸦喝水它的窝 它在灌木丛旁邂逅 一只令它心仪我都妈 难道你手不会痛吗 我叫我爸辈子痛 一壶好酒 再来一碗
 - 是谁 娘子她么在 除有是直 全不了送 恨我不懂 说了没用 我该已好开妈 后知后觉 又过了一个秋 后知后觉 我该好好生活 我该好好生活 不知不觉 你已经离开我 不知不觉 我跟了这节奏 后知后觉 又过了一个秋
epoch 300, perplexity 1.638817, time 0.70 sec
 - 我是谁你   我 泪的我 印分这么 不使我不抽经猜透看透不想 你远么 爱步两人 有如果动 这切落空 在隐海中 你一没痛 说我不懂 说了没用 他的笑容 有何己同 在你海中 你一定受 恨我不懂 说了没用 他的笑
 - 是谁 我满 你不下我每能 难成你的形状 随风跟著我 一口一人吃掉 又北哈怯的我 相思寄红豆 相思寄红豆无能为力的在人海中漂泊心伤透 娘子她人在江南等我 泪不休 语沉默娘子她人在江 我来轻的叹写在西元前 深
epoch 400, perplexity 1.487666, time 0.60 sec
 - 我是谁 可道现迷不会痛吗 其物我回家就着 但杰个人已经不是我 没有一九四三 泛黄的春联还残 牧了有没有 我马儿有些瘦 天涯尽头 满脸风霜落寞 近乡情怯的我 相思寄红豆 相思寄红豆无能为力的在人海中漂泊心伤透
 - 是谁 我想 你不是 每你我遇见你是一场悲剧 我想我这辈子注定一个人演戏 最后再一个人慢慢的回忆 没有了木板的屋内还弥漫 姥姥当年酿的